<a href="https://colab.research.google.com/github/horasan/eng_to_sql_ner/blob/main/NER_B_11_csv_to_json_schema_convertor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# read data from google drive
drive.mount('/content/drive')
FOLDER_PATH = "NER_for_SQL"
FULL_PATH = "/content/drive/My Drive/Colab Notebooks/" + FOLDER_PATH + "/"

Mounted at /content/drive


In [3]:
schema_input_csv = "NER-B-1_schema_info_trs.csv"
schema_relationship_json_output = "NER-B-1_schema_info_trs.json"

In [2]:
import csv
import json
from collections import defaultdict

def csv_to_schema_json(csv_path, json_output_path):
    schema = {
        "tables": [],
        "relationships": []
    }

    table_dict = defaultdict(list)
    seen_relationships = set()

    with open(csv_path, newline='', encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        # Normalize headers
        fieldnames = [name.strip().upper() for name in reader.fieldnames]
        reader.fieldnames = fieldnames

        for row in reader:
            row = {k.strip().upper(): (v.strip() if v else "") for k, v in row.items()}

            table = row.get('TABLE_NAME')
            column = row.get('COLUMN')
            entity = row.get('BUSINESS_ENTITY')
            related_table = row.get('RELATED_TABLE')
            related_column = row.get('RELATED_COLUMN')

            if table and column:
                col_entry = {"name": column}
                if entity:
                    col_entry["business_entity"] = entity
                table_dict[table].append(col_entry)

            if table and column and related_table and related_column:
                rel_key = (table, column, related_table, related_column)
                if rel_key not in seen_relationships:
                    schema["relationships"].append({
                        "from_table": table,
                        "from_column": column,
                        "to_table": related_table,
                        "to_column": related_column
                    })
                    seen_relationships.add(rel_key)

    for table_name, columns in table_dict.items():
        schema["tables"].append({
            "table_name": table_name,
            "columns": columns
        })

    with open(json_output_path, 'w', encoding='utf-8') as f:
        json.dump(schema, f, indent=2, ensure_ascii=False)

    print(f"✅ Schema JSON saved to: {json_output_path}")


In [4]:
schema_input_csv = "NER-B-1_schema_info_trs.csv"
schema_relationship_json_output = "NER-B-1_schema_info_trs.json"
csv_to_schema_json(FULL_PATH + schema_input_csv, FULL_PATH + schema_relationship_json_output)


✅ Schema JSON saved to: /content/drive/My Drive/Colab Notebooks/NER_for_SQL/NER-B-1_schema_info_trs.json
